<font size="6">Линейный классификатор </font>

# Ограничения алгоритма k-nearest neighbors (kNN)



Продолжим с того места, где мы остановились в предыдущей лекции - классификация методом ближайших соседей (kNN). 

**Метрики расстояния**

В практических заданиях EX01 мы использовали kNN для классификации изображений CIFAR10 (набор фотографий разделенных на 10 классов). Мы выяснили, что точность метода классификации с помощью kNN с использованем расстояний L1 (*Manhatten distance* - сумма абсолютных разностей между пикселями) оставляет желать лучшего, и попробовали применить L2 (*Euclidian distance*) в надежде эту точность повысить. С метриками L1 и L2 мы будем сталкиваться часто: и в качестве loss функции, и в качестве регуляризации, поэтому познакомиться с ними полезно. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/l1_manhattan_and_l2_euclidian_distance.png" width="600">

## kNN для классификации



<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/knn_for_classification.jpg" width="450">

На практике, метод ближайших соседей для классификации используется крайне редко. Давайте разберемся почему.
Проблема заключается в следующем: предположим, что точность классификации нас устраивает. Теперь давайте применим kNN на больших данных (e.g. миллион картинок). Для определения класса каждой из картинок, нам нужно сравнить ее со всеми другими картинками в базе данных, такие расчеты, даже в супер оптимизированном виде, занимают много времени. Мы хотим, чтобы обученная модель работала быстро.

Тем не менее, метод ближайших соседей используется в других задачах, где без него обойтись сложно. Например, в задаче распознавания лиц. Представим, что у нас у нас есть большая база данных с фотографиями лиц (например, по 5 разных фотографий всех сотрудников, которые работают в офисном здании, как на примере выше) и есть камера, установленная на входе в это здание. Мы хотим узнать, кто и во сколько пришел на работу. Для того чтобы понять кто прошел перед камерой, нам нужно зафиксировать лицо этого человека и сравнить его со всеми фотографиями лиц в базе. В такой формулировке мы не пытаемся определить конкретный класс фотографии, а всего лишь определяем “похож-не похож”. Мы смотрим на k ближайших соседей и, например, если из k соседей, 5 - это фотографии Джеки Чана, то, скорее всего, под камерой прошел именно он. В таких случаях kNN метод вполне полезен. Похожим образом работает и поиск дубликатов в базах данных.

Примеры эффективной реализации метода на основе kNN:
* [Facebook AI Research Similarity Search](https://github.com/facebookresearch/faiss) – разработка команды Facebook AI Research для быстрого поиска ближайших соседей и кластеризации в векторном пространстве. Высокая скорость поиска позволяет работать с очень большими данными – до нескольких миллиардов векторов.
* Алгоритм поиска ближайших соседей [Hierarchical Navigable Small World](https://arxiv.org/abs/1603.09320). 

## Практические аспекты работы с классификаторами

### Кросс-валидация

Каждая модель имеет как ряд **параметров**, которые она меняет в процессе обучения (например, веса модели), так и ряд **гиперпараметров**, которые влияют на то, каким способом модель меняет параметры в процессе обучения. 

В случае kNN параметры, строго говоря, отсутствует - модель просто запоминает объекты обучающей выборки. Особо упорные могут считать их параметрами. 

А вот гиперпараметры есть, даже несколько групп. Какие? 

1. число соседей 
2. функция, которой считаем расстояние между объектами (L2=eucledian, L1=manhattan)
3. веса, с которыми складываем метки ближайших соседей
4. признаки! (но об этом с вами поговорим позже)
5. сама модель - мы могли выбрать не kNN, а нагуглить что-нибудь другое


Не понятно, насколько результат зависит от того, как нам повезло или не повезло с разбиением данных на обучение и тест. Может оказаться так, что для конкретного разбиения хорошо выбрать k=5, а для другого - k=7. 

Кроме того, опять же - фактически, мы сами выступаем в роли модели, которая учит гиперпараметры (а не параметры) под видимую ей выборку. 





Получается, что если подбирать гиперпараметры модели на *train set*, то:
1. можно переобучитьcя, просто на более "высоком" уровне. Особенно если гиперпараметров у модели много и все они разнообразны
2. нельзя быть уверенным, что выбор параметров не зависит от разбиения на обучение и тест 

Поэтому мы:

1. подбираем гиперпараметры моделей на отдельном датасете, называмым валидационным. Получаем мы его разбиением обучающего датасета на собственно обучающий и валидационный 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L02/split_dataset_for_train_val_test.png" width="400">

2. чаще всего делаем несколько таких разбиений по какой-то схеме, чтобы получить уверенность оценок качества для моделей с разными гиперпараметрами - **кросс-валидация**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_validation_on_train_data.png" width="500">

Часто применяется следующий подход, называемый [K-Fold кросс-валидацией](https://scikit-learn.org/stable/modules/cross_validation.html):

Берется тренировочная часть датасета, разбивается на части - блоки. Дальше мы будем использовать для проверки первую часть (Fold 1), а на остальных учиться. И так последовательно для всех частей. В результате у нас будут информация о точности для разных фрагментов данных и уже на основании этого можно понять, насколько значение этого параметра, который мы проверяем, зависит или не зависит от данных. То есть если у нас от разбиения точность при одном и том же К меняться не будет, значит мы подобрали правильное К. Если она будет сильно меняться в зависимости от того, на каком куске данных мы проводим тестирование, значит, надо попробовать другое К и если ни при каком не получилось - то это такие данные.  


Подберем параметры для модели с помощью **GridSearchCV**.

GridSearchCV – это инструмент для автоматического подбирания параметров для моделей машинного обучения. GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Download dataset 
X, y = load_iris(return_X_y = True) 
# Z-normalize data
X = StandardScaler().fit_transform(X)
X_train_norm, X_test_norm, Y_train, Y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from IPython.display import clear_output
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
"""
Parameters for GridSearchCV:
estimator — model
cv — num of fold to cross-validation splitting 
param_grid — parameters names
scoring — metrics 
n_jobs - number of jobs to run in parallel, -1 means using all processors.
"""
model = GridSearchCV(
    estimator=KNeighborsClassifier(),
    cv=KFold(3, shuffle=True, random_state=42),
    param_grid={
        "n_neighbors": np.arange(1, 31),
        "metric": ["euclidean", "manhattan"],
    },
    scoring="accuracy",
    n_jobs=-1,
)
model.fit(X_train_norm, Y_train)
clear_output()

Выведем лучшие гиперпараметры для модели, которые подобрали:

In [ ]:
print("Metric:", model.best_params_["metric"])
print("Num neighbors:", model.best_params_["n_neighbors"])


Объект GridSearchCV можно использовать как обычную модель

In [ ]:
from sklearn.metrics import balanced_accuracy_score

Y_pred = model.predict(X_test_norm)
print(f"Percent correct predictions {np.round(accuracy_score(y_pred=Y_pred, y_true=Y_test)*100,2)} %")
print(f"Percent correct predictions(balanced classes) {np.round(balanced_accuracy_score(y_pred=Y_pred, y_true=Y_test)*100,2)} %")

**Можно ли делать только кросс-валидацию (без теста)?**


Нет, нельзя. Кросс-валидация не до конца спасает от подгона параметров модели под выборку, на которой она проводится. Оценка конечного качества модели должно производиться на отложенной тестовой выборке. Если у вас очень мало данных, можно рассмотреть [вложенную кросс-валидацию](https://weina.me/nested-cross-validation/), Речь об этом пойдет позже, в последующих лекциях. Но даже в этом случае придется анализировать поведение модели, чтобы показать, что она учит что-то разумное. Кстати, вложенную кросс-валидацию можно использовать, чтобы просто получить более устойчивую оценку поведения модели на тесте.

# Линейный классификатор

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/use_knn_to_comare_imgs_to_find_similar.jpg" width="800">

Давайте подумаем, как избавиться от проблемы со скоростью K-Nearest Neighbors. Реализуя метод ближайшего соседа, мы сравнивали одно изображение со всеми, при этом мы предполагали, что изображения из одного класса будут чем-то похожи друг на друга, и поэтому разница будет меньше. Метод как-то работал. 

Чтобы ускорить этот процесс, мы можем взять весь блок изображений (справа), который относится, например, к машинам и усредним. Таким образом, получим некоторый шаблон для класса “автомобиль”. Скорее всего, работать такой подход будет не слишком здорово, но зато вместо тысяч изображений (в данном случае 50000) появится одно. Возможно, это поможет сильно сэкономить время.

## Переход к сравнению с шаблоном

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/one_template_for_each_class_for_classification.jpg" width="450">

Идея в следующем: вместо того, чтобы сравнивать каждое изображение со всеми остальными по очереди, будем сравнивать изображения с шаблоном для каждого класса. Их будет всего 10 для данного датасета. Этот подход позволит радикально увеличить скорость. 

Проверим, что получится из этой идеи. 


скачиваем CIFAR10 в архиве

In [ ]:
from IPython.display import clear_output 
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzf cifar-10-python.tar.gz
clear_output()

Загрузили архив в память целиком, используя для этого фрагмент кода с сайта CIFAR10. 

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict

path = '/content/cifar-10-batches-py/'

X_train = np.zeros((0, 3072))
Y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"{path}/data_batch_{i}")
    X_train = np.append(X_train, np.array(raw[b"data"]), axis=0)
    Y_train = np.append(Y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle(f"{path}/test_batch")
X_test = np.array(test[b"data"])
Y_test = np.array(test[b"labels"])

labels_eng = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train shape : {X_test.shape}, Y_test shape: {Y_test.shape}")

In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

plt.figure(figsize = (20.0, 20.0))  
for i in range(10):  # for all classes(0 to 9)
    # get indexes for each class 
    label_indexes = np.where(Y_train == i)[0]  
    index = np.random.choice(label_indexes)
    # reshape for vizualization  
    img = X_train[index].reshape(
        32, 32, 3, order="F"
    )  
    img = np.rot90(img, k=3)  
    plt.subplot(1, 10, i + 1)  
    plt.title(labels_eng[i])  
    plt.axis("off")  
    imshow(img.astype("uint8"))  

Для начала создадим шаблоны и визуализируем их.


In [ ]:
templates = []  
meta = unpickle(f"{path}/batches.meta")  # load data
labels = meta[b"label_names"]  

for i in range(len(labels)):
    imgs = X_train[Y_train == i] # select images of i-th class
    mn = np.mean(imgs, 0)  # mean values for all images for each class
    templates.append(mn)

In [ ]:
plt.figure(figsize = (20.0, 2.0))

def show_templates(templates, labels):
    for i, template in enumerate(templates):
        img = template.reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
        plt.subplot(1, len(labels), i + 1)
        plt.title(labels_eng[i])
        plt.axis("off")
        imshow(img)

show_templates(templates, labels)  

Напишем классификатор, который будет сравнивать изображение с шаблоном, который генерируется во время обучения.

In [ ]:
class TemplateBasedClassifier():
    def __init__(self):
        self.templates = []  
        meta = unpickle(f"{path}/batches.meta") 
        self.labels = meta[b"label_names"]  

    def fit(self, X_train, Y_train):
        self.templates = []
        for label_num in range(len(self.labels)):  
            imgs = X_train[Y_train == label_num] 
            mn = np.mean(imgs, 0)  
            self.templates.append(mn)  

    def forward(self, x):
        distances = np.sum(
            np.abs(self.templates - x), axis=1)  # compute distance score
        return np.argmin(distances)  # return minimum score index

Теперь у нас есть 10 шаблонов, с которыми мы будем сравнивать изображение и на основании этого делать предсказание. На этих шаблонах можно увидеть очертания объектов.

Сделаем предсказание и замерим время

In [ ]:
model = TemplateBasedClassifier()  
model.fit(X_train, Y_train)  

In [ ]:
show_templates(model.templates,labels)

In [ ]:
import time

def validate(model, X_test, Y_test, noprint=False):
    correct = 0  # num of correct predictions
    for i, img in enumerate(X_test):  # for all images
        index = model.forward(img)  # predict class
        correct += (
            1 if index == Y_test[i] else 0
        )  
        if noprint is False:  
            if i > 0 and i % 1000 == 0:  
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  
    return correct / len(Y_test)  

start = time.perf_counter()  
accuracy= validate(model, X_test, Y_test)  
tm = time.perf_counter() - start 
print(
    "\nAccuracy {:.2f} Train {:d} /test {:d} in {:.1f} sec. speed {:.2f} samples per second.".format(
        accuracy, len(X_train), len(X_test), tm, len(X_test) / tm
    )
)

Если бы мы обучали KNN на таком количестве данных, это заняло бы порядка 15 мин.



##  Переход к весам
----
Умножение вместо вычитания - для чего?


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/compute_scalar_product_with_compare_with_template.jpg" width="450">

В сравнении с шаблонами, мы вычитали нашу картинку из шаблона и таким образом смотрели, насколько они похожи (L1). Теперь давайте сделаем следующий, пока ничем не обоснованный, ход: оставим всё, как было, но заменим вычитание умножением. Логика в том, что не все пиксели одинаково важны. Вероятно, если на изображении совпадут какие-то пиксели, которые отвечают, например, за глаза кошки, это будет намного важнее, чем фон, который может быть точно таким же у собаки. Здесь будут какие-то важные особенности, которым можно придать больший вес.

Если мы распишем в виде формул наложение шаблона на картинку таким образом с умножением, то получается, что мы скалярно перемножаем два вектора. Подробнее про [скалярное произведение](https://ru.wikipedia.org/wiki/Скалярное_произведение) векторов. Для того, чтобы получить вектор из изображения размерностью 32х32х3, достаточно "выпрямить" его, получим вектор размерностью 1х3072.


###  Математическая запись

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/img_to_vector_to_compute_scalar_product.jpg" width="700">



Обозначим входное изображение как $x$, а шаблон для первого из классов как $w_0$.

Элементы пронумеруем подряд 1,2,3 … $n$. То есть развернем матрицу пикселей изображения в вектор. 

Тогда результат сравнения изображения с этим шаблоном будет вычисляться по формуле: $x[0]*w0[0] + x[1]*w0[1] + … x[n-1]*w0[n-1]$ 




<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/scalar_product_ways_to_use.png" width="800">


Эта простая модель лежит в основе практически всех сложных, которые мы будем рассматривать дальше. Внутри мы будем также пользоваться скалярным произведением.

В дальнейшем мы будем проходить сверточные сети, они работают очень похоже:
мы тоже накладываем шаблон на некоторую матрицу и перемножаем элементы, затем складываем. Единственное отличие – обычно ядро свертки меньше, чем размер самого изображения. 



In [ ]:
import numpy as np

W = np.array([0,0.1,-0.2])
x = np.array([1,2,3])
y = W.dot(x) # 0 + 0.2 -0.6
print(y)

### Регрессия

#### Пример простой линейной регрессии



**Регрессия** - это статистический метод, используемый в финансах, инвестировании и других дисциплинах, который пытается определить силу и характер связи между одной зависимой переменной (обычно обозначаемой **Y**) и рядом других переменных (известных как независимые переменные). В задачах регрессии, мы будем пытаться минимизировать ошибку между предсказанием и истинными данными. 

"Регрессия" происходит от слова "регресс", которое, в свою очередь, происходит от латинского "regressus" - возвращаться (к чему-либо). В этом смысле регрессия - это техника, которая позволяет "вернуться назад" от беспорядочных, трудно интерпретируемых данных к более четкой и осмысленной модели.

Загрузим датасет

In [ ]:
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L02/student_scores.csv
clear_output()

Посмотрим, что там в нем. Видим, что у нас есть два признака - часы и результаты

In [ ]:
import pandas as pd

dataset = pd.read_csv("/content/student_scores.csv")
print(dataset.shape)
dataset.head()

Построим график зависимости одного от другого, а так же отобразим распределения каждой из переменных

In [ ]:
import seaborn as sns
sns.jointplot(data=dataset, x="Scores", y="Hours")
plt.show()

Разделим наши данные на train и test

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, :-1].values # column Hours
Y = dataset.iloc[:, 1].values # column Score

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

Теперь создадим модель для линейной регрессии. Чтобы не писать с нуля, воспользуемся готовой моделью из библиотеки `sklearn`

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() 

И обучим ее

In [ ]:
regressor.fit(X_train, Y_train) 

Посмотрим, что получилось

In [ ]:
X_train.shape

In [ ]:
X_points = np.linspace(
    min(X_train), max(X_train), 100
)  # 100 dots at min to max
Y_pred = regressor.predict(X_points)  

plt.figure(figsize=(8, 5))
plt.plot(X_train, Y_train, "o", label="Scores")
plt.plot(X_points, Y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Теперь сделаем предсказание для тестовой выборки

In [ ]:
Y_pred = regressor.predict(X_test)  

X_points = np.linspace(
    min(X_test), max(X_test), 100
)  
Y_pred = regressor.predict(X_points) 

plt.figure(figsize=(8, 5))
plt.plot(X_test, Y_test, "o", label="Scores")
plt.plot(X_points, Y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Выглядит не плохо

Посчитаем метрики для наших значений


$$ MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$



In [ ]:
from sklearn import metrics

Y_pred = regressor.predict(X_test)

print("Mean Absolute Error: %9.2f" % metrics.mean_absolute_error(Y_test, Y_pred))
print("Mean Squared Error: %10.2f" % metrics.mean_squared_error(Y_test, Y_pred))
print("Root Mean Squared Error: %5.2f" % np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

#### Геометрическая интерпретация 

Теперь, когда мы разобрались с тем, что такое регрессия и с чем ее едят, вернемся к нашим картинкам. Как можно перейти к классификации?

Предположим у нас есть только 2 класса. Как можно использовать регрессию для того, чтобы определить относится ли изображение к классу 0 или к классу 1? В упрощенном варианте, задача будет состоять в том, чтобы провести разделяющую плоскость (прямую) между 2-мя классами. Например, мы можем провести прямую через 0. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_imgs.png" width="270">





Рассмотрим другую ситуацию, в этом случае, мы не можем просто провести прямую через 0. Но можем отступить от 0 на какое-то расстояние и провести ее там. Вспомним, что уравнение прямой это $y=wx+b$, где $b$ - это смещение (*bias*). Соответственно если b != 0, то прямая через 0 проходить не будет, а будет проходить через значение b.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_add_bias.png" width="270">

[Linear Classification Loss Visualization
](http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/)


Если у нас есть несколько классов (несколько шаблонов), мы можем для каждого из них посчитать уравнение $y_{i} = w_{i}x_{i}+b_{i}$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_add_bias_add_multiclasses.jpg" width="400">







На картинке нас интересуют 3 класса. Соответственно, мы можем записать систему линейных уравнений:

\begin{aligned}
y_{0} = w_{0}x_{0} + b_{0} \\
y_{1} = w_{1}x_{1} + b_{1} \\
y_{2} = w_{2}x_{2} + b_{2} \\
\end{aligned}

#### Добавление смещения

Мы их можем собрать в матрицу, тогда получится следующее:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/scalar_product_add_bias.jpg" width="750">

У нас есть матрица коэффициентов, которые мы каким-то образом подобрали, пока ещё не понятно как. Есть вектор $x$, соответствующий изображению. 

Мы умножаем вектор на матрицу, получаем нашу гиперплоскость для четырехмерного пространства в данном случае. Чтобы оно не лежало в 0, мы должны добавить смещение. И мы можем сделать это после, но можно взять и этот вектор смещения (вектор **b**) просто приписать к матрице **W**.

Что будет выходом такой конструкции? Мы умножили матрицу весов на наш вектор, соответствующий изображению, получили некоторый отклик. По этому отклику мы так же, как и при реализации метода ближайшего соседа можем судить: если он больше остальных, то мы предполагаем, что это кошка.


In [ ]:
img = np.array([56, 231, 24, 2])
w_cat = np.array([0.2, -0.5, 0.1, 2.0])
print("Image ", img)
print("Weights ", w_cat)
print("img * w_cat ", img * w_cat)
print("sum ", (img * w_cat).sum()) # dot product
print("Add bias ", (img * w_cat).sum() + 1.1)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/img_to_function_get_scores.jpg" width="600">


Собирая все вместе, получаем какое-то компактное представление, что у нас есть некоторая функция, на вход которой мы подаем изображение, и у нее есть параметры (веса). Пока происходит просто умножение вектора на матрицу, в дальнейшем это может быть что-то более сложное, функция будет представлять какую-то более сложную модель. А на выходе (для классификатора) мы получаем числа, которые интерпретируют уверенность модели в том, что изображение принадлежит к определенному классу.


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/input_img_scalar_product_add_bias_get_scores.jpg" width="450">



Соответственно, эти коэффициенты, которые являются весами модели, надо каким-то образом подбирать. Но прежде чем подбирать коэффициенты, давайте определимся со следующим: как мы будем понимать, что модель работает хорошо или плохо? Ведь она возвращает достаточно абстрактные числа, которые нужно уметь интерпретировать.


# Функция потерь SVM

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/model_predicted_scores_for_10_classes.jpg" width="550">

По аналогии с тем, что мы уже делали, мы можем сравнивать отклик на ключевой класс  (про который нам известно, что он на изображении, так как у нас есть метка этого класса) с остальными. Соответственно, мы подали изображение кошки и получили на выход вектор. Чем больше значение, тем больше вероятность того, что, по мнению модели, на изображении этот класс. Для кошки в данном случае это значение 2.9. Хорошо это или плохо? Нельзя сказать, пока мы не проанализировали остальную часть вектора. Если бы мы могли посмотреть на все значения в векторе, мы бы увидели, что есть значения больше, то есть в данном случае модель считает, что это собака, а не кошка, потому что для собаки значение максимально. 

На основании этого можно построить некоторую оценку. Давайте смотреть на разницу правильного класса с неправильными. В зависимости от того, насколько уверенность в кошке будет больше остальных, настолько будет наша модель хорошей.

Но поскольку нам важна не работа модели на конкретном изображении, а важно оценить ее работу в целом, то эту операцию нужно проделать либо для всего датасета, либо для некоторой выборки, которую мы подаем на вход и подсчитываем средний показатель. Этот показатель (насколько хорошо работает модель), называется функцией потерь, или **loss функцией**. Называется она так, потому что она показывает не то, насколько хорошо работает модель, а то, насколько плохо. 

Дальше будет понятно, почему так удобнее (разница только в знаке). Как это посчитать для всего датасета?

Мы каким-то образом считаем loss для конкретного изображения, потом усредняем по всем изображениям.

Дано: 3 учебных примера, 3 класса. При некотором W баллы f (a, W) = Wx равны:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/model_predicted_scores_for_3_classes.jpg" width="450">

Функция потерь показывает, насколько хорош наш текущий классификатор.

Дан датасет примеров:

$\begin{Bmatrix} (x_i,y_i)  \end{Bmatrix}_{i=1}^N 	$

Где **$x_i$** изображение и **$y_i$** метка (число).

Потери по набору данных — это среднее значение потерь для примеров:

$ L = {1 \over N}\sum_iL_i(f(x_i,W),y_i)$


Снова возьмём наши обучающие примеры **$x_i$** и метки классов **$y_i$**. Представим прогнозы классификатора (оценки) в виде векторов: **$s = f(x_i, W)$**. Для каждого отдельного примера просуммируем значения оценок всех **$y$**, кроме истинной метки **$y_i$**. То есть, мы получим сумму значений всех неправильных категорий. 

Теперь посчитаем разницу между ложными прогнозами и истинной оценкой: **$s_j − s_yi$** ; **$j≠y_i$**. Если истинная оценка больше, чем сумма всех неправильных прогнозов плюс некоторый дополнительный «зазор» (установим его равным 1) — значит, полученный балл для правильной категории намного превосходит любую ошибочную оценку. Это и будет наша функция потерь.

В символьном виде это выглядит так:


$f(n) = \sum_{j\neq y_i}\begin{cases}
  0,  & \mbox{если } s_{y_i}\geq s_j+1\mbox{} \\
  s_j-s_{y_i}+1, & \mbox{если наоборот, то} \mbox{}
\end{cases}$

$=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$





## Вычисление функции потерь SVM 


 Не самая мощная, но достаточно интуитивно понятная loss функция –  SVM loss. 
 
Логика такая: если у нас уверенность модели в правильном классе большая, то модель работает хорошо и loss для данного конкретного примера должен быть равен нулю. Если есть класс, в котором модель уверена больше, чем в правильном, то loss должен быть не равен нулю, а отображать какую-то разницу, поскольку модель сильно ошиблась. При этом есть ещё одно соображение: что будет, если на выходе у правильного и ошибочного класса будут примерно равные веса? То есть, например, у кошки было бы 3.2, а у машины не 5.2, а 3.1. В этом случае ошибки нет, но понятно, что при небольшом изменении в данных (просто шум) скорее всего она появится.
 


То есть модель плохо отличает эти классы. Поэтому мы и вводили некоторый зазор, который должен быть между правильным и неправильным ответом. 

Посмотрим на изображение снизу. У нас есть два класса: фиолетовые треугольники и синие квадраты, разделенные зазором. Также можем увидеть желтые треугольники и квадраты - это ошибочно распознанные классы.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/svm_decision_boundary.png" width="300">


И тоже учитывать его в loss функции: то есть сравнивать его результат для правильного класса не с чистым выходом для другого, а добавить к нему некоторую дельту (в данном случае – 1(единица)). Смотрим: если разница больше 0, то модель работает хорошо и loss равно нулю. Если нет, то мы возвращаем эту разницу, и loss будет складываться из этих индивидуальных разниц.

$f(n) = \sum_{j\neq y_i}\begin{cases}
  0,  & \mbox{если } s_{y_i}\geq s_j+1\mbox{} \\
  s_j-s_{y_i}+1, & \mbox{если наоборот, то} \mbox{}
\end{cases}$

$=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Ниже пример того, как считается loss.


Считаем функцию потерь для 1-ого изображения:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/compute_loss_use_model_scores_for_1_example.jpg" width="650">

Также считаем потери для 2-ого и 3-его изображения:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/compute_loss_use_model_scores_for_2_and_3_examples.jpg" >

Значения losses получились следующие:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/summary_losses_for_3_examples.png" width="450">

Считаем среднее значение loss для всего датасета:

$ L = {1 \over N}\sum_{i=1}^N L_i$

$L={2.9 + 0 + 12.9 \over 3} = 5.27$


SVM loss

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$


Как записать это в коде:

In [ ]:
W = np.random.randn(10, 3072) * 0.0001 # random weights
X = np.random.randn(100, 3072) # fake dataset
Y = np.random.randint(0,10,(100, 1)) # fake labels

In [ ]:
def Li_svm(X, Y, W):
    Y = int(Y)  # to use like a index
    scores = W.dot(X)  
    margins = np.maximum(0, scores - scores[Y] + 1)  # loss
    margins[Y] = 0  
    return np.sum(margins)  


L0 = Li_svm(X[0], Y[0], W)
print("Loss on one image", L0)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/losses_at_set_of_data.jpg" width="900">

Собираем все вместе: считаем loss для всего датасета, усредняя ее.


Как будет выглядеть график этой SVM-loss?

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/svm_plot.gif" width="400">


Функция потерь для ситуации, когда зазор большой, будет равна 0. Она будет меняться только тогда, когда у нас есть ошибка. Причем будет меняться линейно.

Как мы можем использовать эту loss функцию? Допустим, мы знаем, что у нас модель будет работать не очень хорошо (выдает большую ошибку). Можно попытаться найти минимум этой loss функции, то есть подобрать такое W, когда loss функция обратится в 0  (в данном случае), а в общем случае (если какое-то количество данных нельзя обратить в 0), мы можем попытаться найти ее минимум.



# Обновления весов методом градиентного спуска

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/backpropagation_weight_optimization.png" width="750">

Идея в следующем: у нас есть некоторая поверхность, на которой будет точка минимума функции. Мы должны обновлять веса таким образом, чтобы смещаться в сторону этой точки. Loss показывает, как возрастает функция потерь. Так как потери - это плохо, мы должны их минимизировать, то есть мы должны уменьшать веса в сторону, обратную росту этой функции.

Если переходить на случай n-мерного (в данном случае трехмерного) пространства, здесь достаточно очевидна аналогия с поверхностью: у нас есть поверхность земли, которая описывается координатами $x, y, z$. Если мы движемся по этой поверхности, высота $(z)$ будет зависеть от $x, y$. 

$x, y$ - это координаты. Мы можем записать их как вектор. Наша функция будет работать с вектором координат и выдавать скаляр, третью координату. Если в качестве координат мы будем использовать веса нашей модели, а в качестве $z$ - loss, то аналогия станет полной. Наша задача сведется к тому, чтобы найти такой набор весов, при котором значение функции будет минимально. То есть мы окажемся в каком-то минимуме, где ошибка будет минимально возможна для этих данных.

Для того чтобы предположить, где он может находиться, надо понимать, в какую сторону функция растет, а в какую - убывает. Для этого существует производная.

Поскольку у нас функция от нескольких переменных, если мы будем брать от нее производную, у нас получится вектор частных производных, то есть градиент этой функции, который будет показывать, как она меняется в каждом направлении.

## Градиент функции потерь


Градиент — вектор, своим направлением указывающий направление наибольшего возрастания некоторой величины **φ**, значение которой меняется от одной точки пространства к другой (скалярного поля), а по величине (модулю) равный скорости роста этой величины в этом направлении.

Например, если взять в качестве **φ**  высоту поверхности земли над уровнем моря, то её градиент в каждой точке поверхности будет показывать «направление самого крутого подъёма», и своей величиной характеризовать крутизну склона.

Другими словами, градиент — это производная по пространству, но в отличие от производной по одномерному времени, градиент является не скаляром, а векторной величиной.

Для случая трёхмерного пространства градиентом скалярной функции **φ=φ(x,y,z)** координат **(x,y,z)** называется векторная функция с компонентами
$$\frac{\partial\varphi}{\partial x}, \frac{\partial\varphi}{\partial y}, \frac{\partial\varphi}{\partial z}$$.

Если **φ**  — функция **n** переменных **X1...xn**, то её градиентом называется **n**-мерный вектор,
$$\frac{\partial\varphi}{\partial x_1},...,\frac{\partial\varphi}{\partial x_n}$$



компоненты которого, равны частным производным **φ** по всем её аргументам.

Размерность вектора градиента определяется размерностью пространства (или многообразия), на котором задано скалярное поле, о градиенте которого идёт речь.
Оператором градиента называется оператор, действие которого на скалярную функцию (поле) даёт её градиент. Этого оператора иногда коротко называют просто «градиентом».


$W$ - матрица(вектор) весов

$L$ - функция потерь

$\partial W = W_2 - W_1$

$\partial L = L_2 - L_1$


$\frac{\partial L}{\partial W}=\begin{bmatrix}
\frac{\partial L}{\partial W_1} \\
\frac{\partial L}{\partial W_2} \\
... \\
\frac{\partial L}{\partial W_n}
\end{bmatrix}$


Наша задача будет сводиться к тому, что мы будем искать градиент loss функции по весам, которые будут состоять из производной по каждому направлению. Поскольку у нас здесь числа, можно считать этот градиент численно.

## Численный расчет производной

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/gradient_descent_analytical_calculation.png" width="650">



Посчитаем градиент приближенно, воспользовавшись определением (в формуле аргумент обозначен как $x$, у нас же аргументом будет $W$):
На нулевом шаге у нас есть $W_0$ найдем $L_0 = Loss(f(W_0,x))$
Прибавим к первому элементу $W_0$ небольшую величину  $h$ = 0.0001 и получим новую матрицу весов $W_1$ отличающуюся от $W_0$ на один единственный элемент.

Найдем Loss от $\frac {W_1}  {L_1} = Loss(f(W_1,x))$
По определению производной $\frac {dL}{W_0} = \frac{( L_1 - L_0 )}   {h}$

Повторяя этот процесс для каждого элемента из $W$, найдем вектор частных производных, то есть градиент $\frac{dL}{dW}$.

Плюсы:
Это просто. 

Проблемы:

1. Это очень долго, нам придется заново искать значение loss функции для каждого $W_i$.

2. Это неточно, так как по определению приращение $h$ бесконечно мало, а мы используем конкретное пусть и небольшое число. И если мы сделаем его слишком маленьким, то столкнемся с ошибками связанными с округлением в памяти компьютера.


Поэтому данный метод может использоваться как проверочный. 
А на практике вместо него используется **аналитический расчет градиента**.


## Аналитический расчет производной от функции потерь SVM

##### $\color{brown}{\text{Chain-rule }}$



Производная функции $f(g)$:

$$\frac {\delta f} {\delta g}$$

Пусть $g$  на самом деле не просто переменная, а зависит от $h$. Тогда производная от $f$ по $g$ **не меняется**, а производная $f$ по $h$ запишется следующим образом:


$$\frac {\delta f(g(h))} {\delta h} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h}$$

Пусть теперь $h$ зависит от $x$. Все аналогично


$$\frac {\delta f(g(h(x)))} {\delta x} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h} \frac {\delta h} {\delta x}$$

Так можно делать до бесконечности, находя производную сколь угодно сложной функции. И, что важно - мы можем считать градиенты частями - посчитать сначала $f$ по $g$, потом $g$ по $h$....

$$\frac {\delta log(x^2 + 5)} {\delta x}$$

$$h = x^2 + 5$$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac {\delta log(h)} {\delta h} \frac {\delta h} {\delta x}$$ 

$$\frac {\delta log(h)} {\delta h} = \frac 1 h$$

$$\frac  {\delta h} {\delta x} = 2x $$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac 1 {x^2 + 5} \cdot 2x = \frac {2x} {x^2 + 5}$$


##### $\color{brown}{\text{Простые производные }}$



$$x' = \frac {\delta x} {\delta x} = 1$$ 

$$(x^2)' = \frac {\delta x^2} {\delta x} = 2x$$

$$(\log x)'  = \frac {\delta \log x} {\delta x} = \frac 1 x $$

$$(e^x)'  = \frac {\delta e^x} {\delta x} = e^x $$

$$\frac {\delta cf(x)} {\delta x}= c \cdot \frac {\delta f(x)} {\delta x}$$

$$\frac {\delta f(x) + c} {\delta x}= \frac {\delta f(x)} {\delta x}$$

$c$ - константа, не зависящая от $x$

$$ \frac {\delta [f(x) + g(x)]} {\delta x} = \frac {\delta f(x)} {\delta x}  + \frac {\delta g(x)} {\delta x} $$ 


$$\frac {\delta (x^2 + y^3)} {\delta x} = 2x  $$
так как $y$ по отношению к $x$ - константа и мы меняем только $x$

$$\frac {\delta (x^2 + y^3)} {\delta y} = 3y^2  $$
так как $x$ по отношению к $y$ - константа и мы меняем только $y$

$$(e^y)'  = \frac {\delta e^y} {\delta x} = 0 $$

### SVM-Loss

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g1.png" width="750"> 


Распишем формулу для $L_i$ через скалярное произведение.
Затем раскроем сумму.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g2.png" width="750"> 

Считаем производную по первому элементу матрицы **W**. 
Если у нас правильный класс - не первый элемент, то он больше нигде не встретится, кроме как в первой сумме. Поэтому все остальные суммы при подсчете этой частной производной будут константами. Поэтому этот градиент будет равен либо $x_i$,  либо 0 (если эта сумма меньше 0). 

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g3.png" width="750"> 

Аналогично для  $w_i$ частные производные будут равны $x_i$ либо нулю, когда выражение больше меньше 0, 
кроме случая: $j == yi$.  



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g4.png" width="750"> 


Эта ситуация для “правильного изображения”.



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g5.png" width="750">  

Вот финальный результат.
Общие потери считаются по пакету изображений.
“1” в формулах  — это “if”.


In [ ]:
import numpy as np
import random

class LinearClassifier:
  def __init__(self, labels, batch_size, random_state=777):
    self.labels = labels
    self.classes_num = len(labels)
    # Generate a random weight matrix of small numbers
    # Number of weights changed from 3072 to 3073 for implement bias trick
    np.random.seed(random_state)
    self.W = np.random.randn(3073, self.classes_num) * 0.0001
    self.batch_size = batch_size #256
    

  def fit(self, x_train, y_train, learning_rate = 1e-8):
    loss = 0.0
    train_len = x_train.shape[0]
    indexes = list(range(train_len))
    random.shuffle(indexes)

    for i in range(0, train_len, self.batch_size):
      # Split data into batches 
      idx = indexes[i:i+self.batch_size]
      x_batch = x_train[idx]
      y_batch = y_train[idx]

      # Bias trick
      x_batch = np.hstack([x_batch, np.ones((x_batch.shape[0], 1))])

      # Weights update
      loss_val, grad = self.loss(x_batch, y_batch)
      self.W -= learning_rate*grad

      loss += loss_val
    return loss/(train_len)

  def loss(self,x, y):
    current_batch_size = x.shape[0]
    loss = 0.0
    dW = np.zeros(self.W.shape) # Grad
    for i in range(current_batch_size):
      scores = x[i].dot(self.W)
      correct_class_score = scores[int(y[i])]
      above_zero_loss_count = 0
      for j in range(self.classes_num):
        if j == y[i]:
          continue
        margin = scores[j] - correct_class_score + 1 # note delta = 1
        if margin > 0:
          above_zero_loss_count +=1
          loss += margin
          dW[:,j] += x[i] # We summarize it because grad computed over a batch and will de divided by number of examples in this batch
      dW[:,int(y[i])] -= above_zero_loss_count*x[i]
    loss /= current_batch_size
    dW /= current_batch_size
    return loss, dW
      
  def forward(self,x):
    x = np.append(x, 1)
    scores = x.dot(self.W)
    return np.argmax(scores) 


## Выбор шага обучения

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/update_weghts_values.png" width="450"> 

<!-- [Визуализация](https://docs.google.com/file/d/0Byvt-AfX75o1ZWxMRkxrUFJ2ZUE/preview)
 -->



Шаг обучения - некоторый коэффициент, как правило, небольшой, который не позволяет нам двигаться слишком быстро. У нас есть точка, в которую мы хотим попасть. Если мы сделаем слишком большой шаг, то мы ее перескочим (график справа), поэтому надо подобрать шаг, который не позволит ее перескочить, но в то же время, чтобы тот же процесс не шел слишком медленно (как на графике слева)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/learning_rate_optimal_value.png" > 

 

Пока изменения loss функции достаточно большие, сам по себе градиент тоже большой. За счет этого можно двигаться быстро. Когда он будет уменьшаться, мы должны оказаться рядом с этим минимумом, и шаг, который мы выбрали, не должен мешать этому процессу. 

Бывают ситуации, когда шаг можно менять в самом процессе обучения, когда в начале обучения модели шаг большой, потом, по мере того, как она сходится, чтобы более четко найти минимум, шаг можно изменить вручную. Но изначально его нужно каким-то образом подобрать, и это зависит от данных и от самой модели. Это тоже гиперпараметр, связанный с обучением.

## Выбор размера батча



<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/define_size_of_batch.png" width="500"> 

Мы с самого начала говорили о выборках, некоторого количества примеров. В дальнейшем, мы будем называть их батчами. Батч (англ. *batch*) - это некоторое подмножество обучающей выборки фиксированного размера.

При этом было не очень понятно, чем они мотивированны. Точнее, мы мотивировали это тем, что у нас много данных, и мы не сможем их обработать все, и это правда. Даже если мы сможем загрузить все данные в память, нам нужно будет загрузить их и использовать при расчете, в том числе градиента. Это ещё более затратно. 

Зачем батчи нужны при рассчете loss? Если мы посчитаем loss по одному изображению, скорее всего он будет очень специфичен, и это движение, которое произойдет, будет направлено в сторону минимума, потому что по этому конкретному изображению мы улучшим показатели,что не отражает обобщения всех данных, поэтому мы используем батчи.

Нас в первую очередь интересует более общее направление, которое будет работать на большинстве наших данных. Поэтому если мы будем оптимизировать модели, исходя из одного элемента данных, путь будет витиеватый, и процесс будет происходить достаточно долго. Если же данные не помещаются в память, то **можно использовать батч того размера, который у нас есть**. Можно загрузить в память и считать по батчам градиент. В этом случае спуск будет более плавным, чем по одному изображению. 

Также при использовании всего датасета тоже есть свои минусы. **Не всегда загрузка всего датасета приводит к увеличению точности**. 

Подробнее: 
* [Методы оптимизации нейронных сетей
](https://habr.com/ru/post/318970/)
* [Обучение нейронной сети](https://alxmamaev.medium.com/deep-learning-на-пальцах-часть-1-341cfe021ef9)


## Регуляризация

Наша модель может переобучаться - она может просто поднастроить веса по тренировочную выборку. Особенно это характерно для больших моделей, когда число весов приближается к числу параметров. 






<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_regularisation.jpg" width="700"> 


Регуляризация Тихонова = L2 Regularization = weights decay

Идея состоит в том, что мы можем наложить некоторое требование на сами веса. Дело в том, что можно получить один и тот же выход модели при разных весах (выход модели соответствует умножению весов на $x$), при разных $w$ выход может быть идентичен. При этом если мы говорим о том, что эти параметры задают некоторую аппроксимацию нашей целевой функции, то аппроксимировать функцию можно двумя способами:
1. Использовать все имеющиеся данные и провести ее строго через все точки, которые нам известны; 
2. Использовать более простую функцию: (в данном случае линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым общим закономерностям, которые у них есть. 

Характерной чертой переобучения является второй сценарий, и сопровождается он, как правило большими весами. Введение L2-регуляризации приводит к тому, что большие веса штрафуются и предпочтение отдается решениям, использующим малые значения весов. 

Модель может попробовать схитрить и по-другому - использовать все веса, все признаки, даже незначимые, но с маленькими коэффициентами. С этим L2-loss поможет хуже, так как он не сильное штрафует мелкие веса. Результатов его применения - малые значения весов, которые использует модель

В этом случае на помощь приходит L1-loss, который штрафует вес за сам факт отличия его от нуля. Но и штрафует он все веса одинаково. Результат его применения - малое число весов, которые использует модель в принципе. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/losses.gif" alt="alttext" style="width: 500px;"/>


Это лоссы можно комбинировать - получится Elastic Net


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-regularisation2.png" width="700"> 


Берем сумму всех весов по всей матрице $w$, и добавляем ее к loss. Соответственно, чем больше будет эта сумма, тем больше будет суммарный loss. 


В дальнейшем проблема с переобучением будет вставать довольно часто. Методов регуляризации модели существует достаточно много. Этот — один из базовых, который будет использоваться практически во всех отпимизаторах. 

# Функция потерь Кросс-энтропия 

[Отличное видео от Statquest про энтропию](https://www.youtube.com/watch?v=YtebGVx-Fxw)



На Cross-entropy Loss построены практически все модели, про которые будет идти речь.

Это оценка результата и обновление весов с использованием градиента. У нас один слой, поэтому мы достаточно легко посчитали от него градиент. Если слоев будет много, сделать это будет сложней.

Рассмотрим ещё одну loss-функцию. 

SVM loss хороша тем, что интуитивно понятно, как ее посчитать. Но проблема состоит в том, что модель выдает некоторые числа, которые сами по себе невозможно интерпретировать. Сами по себе выходы модели мало что означают. Было бы неплохо, если бы мы сразу, глядя на выход модели для кошки, могли бы их как-то интерпретировать, не просматривая остальные веса. Хорошо бы, чтобы модель выдавала не какую-то абстрактную уверенность, а **вероятность** того, что по ее мнению, на картинке изображена кошка.


## Переход к вероятностям

**Softmax**

[Видео от StatQuest, которое объясняет Softmax](https://www.youtube.com/watch?v=KpKog-L9veg)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/scores_to_probability.png" width="750">

Перейти к вероятностям мы сможем, проведя с весами некоторые не очень сложные математические преобразования. 

На слайде выше показано, почему выходы модели часто называют [logit’ами](https://en.wikipedia.org/wiki/Logit). Если предположить, что у нас есть некая вероятность, от которой мы берем такую функцию (logit), то она может принимать значения от нуля до бесконечности. Мы можем считать, что выходы модели - это logit’ы. 




Например, мы могли бы просто взять индекс массива, в котором значение (logit) максимально. Предположим, что наша сеть выдала следующие значения:

In [ ]:
import numpy as np

logits = [
          5.1, # cat
          3.2, # car
          -1.7, # frog
]

Тогда, чтобы узнать какой класс наша сеть предсказала, мы могли бы просто взять `argmax` от наших `logits`

In [ ]:
print('Predicted class = %i (Cat)' % (np.argmax(logits)))

Но от argmax нельзя посчитать градиент, так как производная от константы равна 0. Соответственно, если бы мы вставили производную от argmax в градиентный спуск, мы бы получили везде нули, и соответственно, наша сеть бы вообще ничему не научилась

А мы бы хотели получить не logit’ы, а настоящую вероятность на выходе модели. Да еще и таким образом, что бы от наших вероятностей можно было бы посчитать градиент. Для этого мы можем применить к нашим логитам функцию **Softmax**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/linear_classifier_softmax.jpg" width="1300">

Мы можем провести над логитами операцию экспоненцирования, то есть число Эйлера (2.71828) **возвести в степень**, соответствующую этому выходу. В результате, мы получим вектор, гарантировано неотрицательных чисел (потому что мы ввели положительное число в степень, пускай даже отрицательную, то есть выходы могут быть маленькие, но всегда положительные). 



Дальше, чтобы можно было интерпретировать эти числа как вероятности, их сумма должна быть равна единице. Мы должны их нормализовать, то есть **поделить на сумму**. Это преобразование называется **Softmax функцией**.

**Получаются вероятности**, то есть числа, которые можно интерпретировать таким образом.

$\text{Softmax}_\text{кошка} = \frac{e^{5.1}}{e^{5.1} + e^{3.2} + e^{-1.7}}$

In [ ]:
def softmax(logits):
    return np.exp(logits)/np.sum(np.exp(logits))

print(softmax(logits))
print('Sum = %.2f' % np.sum(softmax(logits)))

Можно обратить внимание, что Softmax, никоим образом не поменял порядок значений. Самому большому logit'у соответствует самая большая вероятность, а самому маленькому, соответственно самая маленькая

**Cross-entropy / log loss**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_entropy_plot_loss_with_probability.png" width="800">


Теперь, с использованием такого вектора можно определить другую loss функцию. Если не вникать в детали, можно **взять от нее логарифм**. Тогда loss от такого выхода будет выглядеть вот таким образом:

$ L_i =  -log(\frac{e^{s_{y_i}}}{\sum_j e^{s_j}})$

как нормализованный выход от верного класса на сумму остальных, к которому добавили минус. 

Получится график loss (слева). Его плюс заключается в том, что у него нет участка с плато, практически по всей длине функция получается гладкой, с хорошими мощными производными. Когда loss большой, производная тоже большая, и за счет этого можно быстро обучать модель. Для кусочно-линейной функции потерь она же равна константе.

До тех пор, пока модель не будет работать с точностью 100%, то есть пока все остальные выходы не станут нулевыми, мы можем продолжить обучение, даже если у нас нет явной регуляризации.

Осталось выяснить, почему такая простая на вид функция **называется так сложно** - Кросс-энтропия. 



## Определение Кросс-энтропии


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/define_cross_entropy_kulbak_shennon.png" >

Если мы рассматриваем выходы нашей модели как вероятности, то мы можем их сравнивать. У нас есть номер правильного класса. Соответственно, можно сказать, что вероятность этого класса - единица, а всех остальных 0, и получить, таким образом, вероятностное распределение. Выход модели тоже можно интерпретировать как вероятности (тоже можно получить вероятностное распределение). И для работы с этими распределением есть некоторый математический аппарат, который основан на понятии энтропии, который ввел Клод Шеннон.


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/efficient_way_to_send_signals.png" >


Идея в следующем: у нас есть некоторые данные, которые мы передаем по каналу связи. Например, у нас есть метеостанция, которая сообщает прогноз погоды. Допустим, она может передавать 8 вариантов прогноза. Мы в каждый момент времени получаем от нее сообщение. Предположим, что потребуется 3 бит, чтобы передавать это сообщение. 

Допустим, мы знаем некую вероятность, с которой в нашем регионе может наступить хорошая либо плохая погода. То есть мы знаем вероятностное распределение, по которому у нас, допустим, солнечное место, где почти не бывает бури. Если мы знаем об этом, мы можем сэкономить на передаче информации. Мы можем закодировать наиболее вероятные сообщения двумя битами, причем придумать такие коды, чтобы они почти не пересекались. Они будут начинаться с нуля и передавать два бита вместо трех. Таким образом, можно сэкономить на передачи информации. То есть идея такая: **если мы знаем, что события маловероятны, мы можем кодировать их более длинной цепочкой, а более вероятные события - более короткими цепочками**. В этом случае в среднем количество информации, которое можно передать, сократится.

**Формула Шеннона позволяет посчитать, насколько сильно мы можем сэкономить для конкретного вероятностного распределения**. То есть если подставить эти данные в формулу, то получим 2,23.

На практике реализовать это, используя биты, возможно, не выйдет, но это свойство данного вероятностного распределения можно посчитать. Оно несёт в себе некоторое количество полезной информации, соответствующее этой энтропии.

**Что же такое кросс-энтропия?**

Давайте представим, что мы каким-то образом начали кодировать эту информацию, прошили погодную станцию так, что она может эту кодированную информацию передавать, а потом перенесли в другой регион, где ситуация противоположная. И мы кодируем события, которые стали частыми, длинными цепочками, а маловероятные - короткой, и у нас в этом случае явно передается больше информации, чем мы могли бы, если бы сделали все банально. 

Кросс-энтропия позволяет посчитать, насколько большая будет потеря в данном случае. То есть это некий **способ сравнить между собой вероятностные распределения**.

Что, собственно говоря, отображено в формуле, потому что мы считаем кросс-энтропию по нашему вектору $P$ и $Q$. Вектор $P$ состоит из нулей и единиц, соответственно, во всех случаях, кроме одного, это выражение будет нулевым, кроме одного случая с единицей. Случай с единицей соответствует нашему правильному классу. Поэтому сумма исчезает, остаются логарифм и вектор, а вероятность для правильного класса мы считаем как Soft max. Отсюда название кросс-энтропия.  


(В теории информации кросс-энтропия между двумя распределениями вероятностей $p$ и $q$  по одному и тому же базовому набору событий измеряет среднее число битов, необходимых для идентификации события, взятого из набора, если схема кодирования, используемая для набора, оптимизирована для оценочного распределения вероятностей $q$, а не для истинного распределения $p$.)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/logits_to_scores_to_probabilitys.png" width="900">

Аналогичное обоснование: есть понятие дивергенция, которая позволяет оценить расхождение между вероятностными распределениями, которая нам здесь и нужна. Есть дивергенция **Кульбака-Лейблера**, которая выражается через кросс-энтропию, а энтропия от нашего вектора нулевая. Поэтому фактически здесь кросс-энтропия равна дивергенции, которая показывает, насколько не похожи два распределения. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/softmax_formula.jpg" width="800">


Эта функция очень популярна, она используется при обучении реальных моделей на практике. Поскольку внутри находится Softmax, часто ее называют Softmax-классификатором, что, строго говоря, не совсем верно. Здесь функция потерь - кросс-энтропия, Softmax же просто используется внутри нее.


##### $\color{brown}{\text{Градиент функции потерь Кросс-энтропии }}$




[Cross Entropy Loss](http://machinelearningmechanic.com/deep_learning/2019/09/04/cross-entropy-loss-derivative.html)

$$ -L = \sum_i y_i \log p_i = \sum_i y_i \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

$$s_{y_i} = w_i x$$

$$ \dfrac {\delta L} {\delta w_i} = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} $$

$$\dfrac {\delta s_{y_i}} {\delta w_i} = x$$

Только один $y_k = 1$


$$ -L = y_k \log p_i = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

i = k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_i}} - \log  \sum_j e^{s_{y_j}}  = s_{y_i} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = 1 - \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = 1 - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i - 1 $$

$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = (p_i - 1) x $$

i != k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_k}} - \log  \sum_j e^{s_{y_j}}  = s_{y_k} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} =  \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i $$
$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = p_i x $$



In [ ]:
x = np.array([ # batch of 2 vector with 4 elements
              [1,2,3,4], 
              [1,-2,0,0]
            ]
             )
W = np.random.randn(3,4) # 3 class

logits = x.dot(W.T)
print("Scores(Logits)",logits,"\n")

probs = softmax(logits)
print("Probs",probs,"\n")

#probs = np.random.randn(1,3)
y = [0,1] # cat
probs[np.arange(1),y] = -1
dW = x.T.dot(probs)
print("Grads",dW)


## Практическое вычисление SoftMax

Когда мы возводим число в степень, может получиться очень большое число и произойти переполнение памяти. Неизвестно, какие будут у модели выходы. 

In [ ]:
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

f = np.array([123, 456, 789])
p = np.exp(f) / np.sum(np.exp(f))
print(p)

Поэтому мы можем вычесть из каждого $s_i$ положительную константу, чтобы уменьшить значения экспонент. В качестве константы можно выбрать максимальный элемент этого вектора, тогда у нас гарантированно не будет очень больших чисел, и такой способ будет работать более стабильно.

$$M = \max_j s_{y_{j}}$$
$$s^{new}_{y_{i}}  = s_{y_{i}} - M $$

$$ \dfrac {e^{s^{new}_{y_{i}}}} {\sum_j e^{s^{new}_{y_{j}}}}  = \dfrac {e^{s_{y_{i}} - M }} {\sum_j e^{s_{y_{j}} - M }} = \dfrac {e^{s_{y_{i}}}e ^ {-M}} {\sum_j e^{s_{y_{j}}} e ^ {-M}} = \dfrac {e ^ {-M} e^{s_{y_{i}}}} {e ^ {-M} \sum_j e^{s_{y_{j}}} } = \dfrac { e^{s_{y_{i}}}} { \sum_j e^{s_{y_{j}}} }$$

In [ ]:
f = np.array([123, 456, 789])
f -= f.max()
p = np.exp(f) / np.sum(np.exp(f))
print(f, p)